# Засоби підготовки та налізу даних
## Лабораторна робота №4
### ФБ-22 Лаптєв Денис


#### Підготовка середовища

In [103]:
import pandas as pd
import numpy as np
import timeit
import seaborn as sns
import matplotlib.pyplot as plt
import urllib.request
from datetime import datetime, time
print ('setup complete')

setup complete


##### Завантаження датасету

In [6]:
url = "https://archive.ics.uci.edu/static/public/235/data.csv"
filename = "data/data.csv"

urllib.request.urlretrieve(url, filename)

('data/data.csv', <http.client.HTTPMessage at 0x2e3e2776d50>)

In [118]:
def load_data():
    url = "https://archive.ics.uci.edu/static/public/235/data.csv"
    filename = "data/data.csv"

    urllib.request.urlretrieve(url, filename)

def create_df():
    df = pd.read_csv("data/data.csv")

    df = df.drop(df.loc[df['Sub_metering_1'] =='?'].index)

    df['Global_active_power'] = df['Global_active_power'].astype(float)
    df['Global_reactive_power'] = df['Global_reactive_power'].astype(float)
    df['Voltage'] = df['Voltage'].astype(float)
    df['Global_intensity'] = df['Global_intensity'].astype(float)
    df['Sub_metering_1'] = df['Sub_metering_1'].astype(float)
    df['Sub_metering_2'] = df['Sub_metering_2'].astype(float)
    df['Sub_metering_3'] = df['Sub_metering_3'].astype(float)


    df.insert(0,"DataTime", pd.to_datetime(df['Date'] + ' ' + df['Time']))
    df = df.drop(['Date', 'Time'], axis=1)

    return df

#load_data()
df = create_df()

C:\Users\Denys\AppData\Local\Temp\ipykernel_14700\1462750726.py:8: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/data.csv")
C:\Users\Denys\AppData\Local\Temp\ipykernel_14700\1462750726.py:21: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df.insert(0,"DataTime", pd.to_datetime(df['Date'] + ' ' + df['Time']))


In [121]:
df.info()
#df.Time.nunique()
#print(df)
#df.to_csv('output.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 2049280 entries, 0 to 2075258
Data columns (total 8 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   DataTime               datetime64[ns]
 1   Global_active_power    float64       
 2   Global_reactive_power  float64       
 3   Voltage                float64       
 4   Global_intensity       float64       
 5   Sub_metering_1         float64       
 6   Sub_metering_2         float64       
 7   Sub_metering_3         float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 140.7 MB


#### Хід роботи

- проаналізувати часові витрати на виконання процедур(профілювання часу виконання)
- зробити висновки щодо ситуацій, в яких має сенс віддати перевагу тій чи іншій структурі даних. Висновки оформити звітом із зазначеним часом виконання та оцінкою по 5-бальній шкалі зручності виконання операцій відбору.
- залишити лише ті спостереження, в яких немає порожніх спостережень (порожні значення – пусті поля між роздільником – ? – 28.04.2007, як приклад).

1. Обрати всі домогосподарства, у яких загальна активна споживана потужність перевищує 5 кВт.

In [128]:
def fun_1_pd(df):
    result_df = df[(df['Global_active_power'] >= 5)]
    return result_df

def fun_1_np(df):
    result_df = df.iloc[np.where(df['Global_active_power'] >= 5)]
    return result_df

Timer = timeit.timeit()
fun_1_np(df)
print("with np:", Timer)

Timer = timeit.timeit()
fun_1_pd(df)
print("with pd:", Timer)

with np: 0.1014428000053158
with pd: 0.09902060001331847


2. Обрати всі домогосподарства, у яких вольтаж перевищую 235 В.

In [129]:
def fun_2_pd(df):
    result_df = df[(df['Voltage'] >= 235)]
    return result_df

def fun_2_np(df):
    result_df = df.iloc[np.where(df['Voltage'] >= 235)]
    return result_df

Timer = timeit.timeit()
fun_2_np(df)
print("with np:", Timer)

Timer = timeit.timeit()
fun_2_pd(df)
print("with pd:", Timer)

with np: 0.11764159999438561
with pd: 0.10286899999482557


3. Обрати всі домогосподарства, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер.

In [198]:
def fun_3_pd(df):
    result_df = df[(df['Global_intensity'].between(19, 20)) & (df['Sub_metering_2'] > df['Sub_metering_3'])]
    return result_df

def fun_3_np(df):
    result_df = df.iloc[np.where((df['Global_intensity'].between(19, 20)) & (df['Sub_metering_2'] > df['Sub_metering_3']))]
    return result_df

Timer = timeit.timeit(); fun_3_np(df); print("with np:", Timer)

Timer = timeit.timeit(); fun_3_pd(df); print("with pd:", Timer)

with np: 0.10507049999432638
with pd: 0.10464370000408962


4. Обрати випадковим чином 5000(00)? домогосподарств (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії.

In [ ]:
def fun_4_pd(df):
    result_df = df.iloc[np.random.choice(df['Voltage'], size = 5000)] #500000. but not even 5000
    avg_1 = result_df['Sub_metering_1'].mean() # mean from pandas
    avg_2 = result_df['Sub_metering_2'].mean()
    avg_3 = result_df['Sub_metering_3'].mean()
    print(f"Sub 1 avg: {avg_1}\nSub 2 avg: {avg_2}\nSub 3 avg: {avg_3}")
    
    return result_df

print(fun_4_pd(df))

5. Обрати ті домогосподарства, які після 18-00 споживають понад 6 кВт за хвилину в середньому, серед відібраних визначити ті, у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення (група 2 є найбільшою), а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини.

In [105]:
def fun_5_pd(df):
    result_df = df[df["DataTime"].dt.time > time(18, 0)]
    return result_df

print(fun_5_pd(df))

                   DataTime  Global_active_power  Global_reactive_power  \
37      2006-12-16 18:01:00                2.624                  0.144   
38      2006-12-16 18:02:00                2.772                  0.118   
39      2006-12-16 18:03:00                3.740                  0.108   
40      2006-12-16 18:04:00                4.928                  0.202   
41      2006-12-16 18:05:00                6.052                  0.192   
...                     ...                  ...                    ...   
2075254 2010-11-26 20:58:00                0.946                  0.000   
2075255 2010-11-26 20:59:00                0.944                  0.000   
2075256 2010-11-26 21:00:00                0.938                  0.000   
2075257 2010-11-26 21:01:00                0.934                  0.000   
2075258 2010-11-26 21:02:00                0.932                  0.000   

         Voltage  Global_intensity  Sub_metering_1  Sub_metering_2  \
37        238.20             